In [1]:
!pip install rasterio

Defaulting to user installation because normal site-packages is not writeable
--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/segment.py", line 249, in split_lines
    for segment in segments:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1283, in render
    renderable = rich_cast(renderable)
  File 

In [2]:
!pip install pyarrow

Defaulting to user installation because normal site-packages is not writeable
--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/segment.py", line 249, in split_lines
    for segment in segments:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1283, in render
    renderable = rich_cast(renderable)
  File 

In [3]:
import tensorflow as tf
#import dlmodels
import tensorflow.keras.models as models
import tensorflow.keras.optimizers as optimizers
import tensorflow.keras.utils as kutils
import sklearn.ensemble
import sklearn.metrics
import rasterio
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [4]:
path = r'./oil_spill_train.parquet'

In [5]:
path

'./oil_spill_train.parquet'

In [6]:
df = pd.read_parquet('oil_spill_train.parquet')

Some EDA

In [7]:
df

,Red,Green,Blue,labels
0,171,171,171,0
1,239,239,239,0
2,175,175,175,0
3,171,171,171,0
4,201,201,201,0
...,...,...,...,...
102604795,89,89,89,0
102604796,78,78,78,0
102604797,106,106,106,0
102604798,103,103,103,0


In [8]:
df.shape

(102604800, 4)

In [9]:
df.head()

,Red,Green,Blue,labels
0,171,171,171,0
1,239,239,239,0
2,175,175,175,0
3,171,171,171,0
4,201,201,201,0


In [10]:
df.describe

<bound method NDFrame.describe of            Red  Green  Blue  labels
0          171    171   171       0
1          239    239   239       0
2          175    175   175       0
3          171    171   171       0
4          201    201   201       0
...        ...    ...   ...     ...
102604795   89     89    89       0
102604796   78     78    78       0
102604797  106    106   106       0
102604798  103    103   103       0
102604799  119    119   119       0

[102604800 rows x 4 columns]>

In [11]:
df.dtypes

Red       uint8
Green     uint8
Blue      uint8
labels    uint8
dtype: object

In [12]:
df.isnull().sum()

Red       0
Green     0
Blue      0
labels    0
dtype: int64

In [13]:
df.isnull()

,Red,Green,Blue,labels
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
...,...,...,...,...
102604795,False,False,False,False
102604796,False,False,False,False
102604797,False,False,False,False
102604798,False,False,False,False


In [14]:
selected_columns = df[['Red','Green','Blue']]
selected_columns

,Red,Green,Blue
0,171,171,171
1,239,239,239
2,175,175,175
3,171,171,171
4,201,201,201
...,...,...,...
102604795,89,89,89
102604796,78,78,78
102604797,106,106,106
102604798,103,103,103


In [15]:
filtered_data = df[df['Red'] > 100]

In [16]:
filtered_data

,Red,Green,Blue,labels
0,171,171,171,0
1,239,239,239,0
2,175,175,175,0
3,171,171,171,0
4,201,201,201,0
...,...,...,...,...
102604790,130,130,130,0
102604792,111,111,111,0
102604797,106,106,106,0
102604798,103,103,103,0


In [17]:
unique_values_count = df['labels'].nunique()
unique_values_count

5

In [18]:
grouped_data_cat = df.groupby('labels')
grouped_data_cat

In [19]:
sum_values = grouped_data_cat.sum()
sum_values

,Red,Green,Blue
labels,,,
0,1.218148e+10,1.218148e+10,1.218148e+10
1,1.133437e+09,1.133437e+09,1.133437e+09
2,1.310951e+08,1.310951e+08,1.310951e+08
3,9.512624e+07,9.512624e+07,9.512624e+07
4,1.261694e+07,1.261694e+07,1.261694e+07


In [20]:
# plt.figure(figsize=(10, 6))  # Adjust the figure size as needed
# plt.bar(grouped_data_cat['labels'] )  # Replace 'mean_column' with the actual column name
# plt.xlabel('labels')
# plt.ylabel('observations')
# plt.title('number of observation per label')
# plt.xticks(rotation=45)  # Rotate x-axis labels if necessary
# plt.show()

In [4]:
new_tdata = pd.read_parquet('balanced_dataset.parquet')

In [5]:
new_tdata

,Red,labels,glcm
0,0.690196,0,0
1,0.811765,0,0
2,0.411765,0,0
3,0.682353,0,3
4,0.639216,0,0
...,...,...,...
2451975775,0.356863,0,0
2451975776,0.388235,0,0
2451975777,0.439216,0,0
2451975778,0.360784,0,0


training the random forest algorithm

In [21]:
X = df.drop('labels', axis=1)
y = df['labels']

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
from sklearn.ensemble import RandomForestClassifier
class_weights = {0: 1, 1: 5, 2: 10, 3: 10, 4: 10}

In [ ]:
# Create a Random Forest Classifier with class weights
rf_model = RandomForestClassifier(class_weight=class_weights)
rf_model.fit(X_train, y_train)

In [ ]:
# param_grid = {
#     'n_estimators': [50, 100, 200],  # Number of trees
#     'max_depth': [None, 10, 20],  # Maximum depth of each tree
#     'min_samples_split': [2, 5, 10],  # Minimum samples required to split an internal node
# }

In [ ]:
# grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, scoring='accuracy', cv=5)
# grid_search.fit(X_train, y_train)

In [ ]:
# 0 - Sea Surface
# 1 - Oil Spill
# 2 - Look-alike
# 3 - Ship
# 4 - Land

In [ ]:
y_pred = rf_model.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, zero_division='warn')

In [ ]:
print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:\n', report)